# Task 3: Clustering and visualisation

We are now going to perform a clustering on the neighbourhoods within Toronto, based upon the types of venues that each neighbourhood contains. As before, retrieve/rebuild the dataframe `df_postcode` from the previous sections.

In [1]:
# Begin by recreating the dataframe

import requests
from bs4 import BeautifulSoup
import pandas as pd
import pgeocode
geo = pgeocode.Nominatim('ca')

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'lxml')

neighbourhoods = []

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_data = table.find_all('tr')
headers = [th.text.replace('\n','').replace(' ', '') for th in table_data[0].find_all('th')]
for n in range(1, len(table_data)):
    neighbourhood = dict()
    for header, td in zip(headers, table_data[n].find_all('td')):
        neighbourhood[header] = td.text.replace('\n','')
    neighbourhoods.append(neighbourhood)

df = pd.DataFrame(neighbourhoods)
empty_rows = df[df['Borough']=='Not assigned'].index
df.drop(empty_rows, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
df_postcode = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
def get_borough(row):
    if row['Neighbourhood'] == 'Not assigned':
        return row['Borough']
    return row['Neighbourhood']

df_postcode['Neighbourhood'] = df_postcode.apply(lambda row: get_borough(row), axis=1)

pcodes = df_postcode['PostalCode'].values
geo_df = geo.query_postal_code(pcodes)
geo_df = geo_df[['postal_code', 'latitude', 'longitude']]
geo_df.columns = ['PostalCode', 'Latitude', 'Longitude']
df_postcode = df_postcode.merge(geo_df, how='left', on='PostalCode')
df_postcode.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389
5,M1J,Scarborough,Scarborough Village,43.7464,-79.2323
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.7298,-79.2639
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.7122,-79.2843
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.7247,-79.2312
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6952,-79.2646


Before we continue we will import some necessary packages, and set some parameters in `pandas` to ensure that we can inspect our dataframes more readily.

In [2]:
import folium
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

We shall restrict ourselves to boroughs contained within Toronto proper.

In [3]:
toronto_data = df_postcode[df_postcode['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6784,-79.2941
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.6693,-79.3155
3,M4M,East Toronto,Studio District,43.6561,-79.3406
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935
5,M4P,Central Toronto,Davisville North,43.7135,-79.3887
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.7143,-79.4065
7,M4S,Central Toronto,Davisville,43.7020,-79.3853
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.6899,-79.3853
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.6861,-79.4025


In order to map out the neighbourhoods, we also need latitude and longitude values for Toronto, which we shall do with `geopy`.

In [4]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent='to_explorer') 
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f"The geographical coordinates of Toronto are ({latitude}, {longitude})")

The geographical coordinates of Toronto are (43.6534817, -79.3839347)


We now plot a map of Toronto containing each of the neighbourhoods contained within `toronto_data`.

In [5]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, bor, neigh in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = f'{neigh}, {bor}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng], 
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

We now make use of the FourSquare API to obtain up to 100 venues within a radius of 500m of each neighbourhood. We first provide some constants, then define a function that will perform a GET request to the API using the longitude and latitude for each neighbourhood, and then combine these into a dataframe.

In [6]:
CLIENT_ID = 'RNQZEFS4EHVDI0CQERFZEHSLY3QYMBBNCRSSNUBGMFKIJBLB' 
CLIENT_SECRET = 'X2DFDPKNEA4TXS2SRMDJQ42IPBTZS45KQ2KYS112LTYHHJF2' 
VERSION = '20180605' 
LIMIT = 100 

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Creating the new dataframe `toronto_venues` using our just defined function.

In [8]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'], 
                                 latitudes=toronto_data['Latitude'], 
                                 longitudes=toronto_data['Longitude']
                                )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [9]:
print(toronto_venues.shape)
toronto_venues.head(15)

(1525, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6784,-79.2941,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.6784,-79.2941,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.6784,-79.2941,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.6784,-79.2941,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.6784,-79.2941,Grover Pub and Grub,43.679181,-79.297215,Pub
5,The Beaches,43.6784,-79.2941,Upper Beaches,43.680563,-79.292869,Neighborhood
6,The Beaches,43.6784,-79.2941,Mullins Irish Pub,43.680348,-79.289370,Pub
7,"The Danforth West, Riverdale",43.6803,-79.3538,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,"The Danforth West, Riverdale",43.6803,-79.3538,Pantheon,43.677621,-79.351434,Greek Restaurant
9,"The Danforth West, Riverdale",43.6803,-79.3538,La Diperie,43.677702,-79.352265,Ice Cream Shop


A check of how many venues are contained within each neighbourhood.

In [10]:
toronto_venues.groupby('Neighbourhood')[['Venue']].count()

,Venue
Neighbourhood,
Berczy Park,91
"Brockton, Parkdale Village, Exhibition Place",38
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",56
Central Bay Street,57
Christie,11
Church and Wellesley,77
"Commerce Court, Victoria Hotel",100
Davisville,23


In [11]:
# Check the number of unique venue categories

print(f"There are {len(toronto_venues['Venue Category'].unique())} unique categories")

There are 217 unique categories


In order to apply K means clustering we need to encode the venue categories; we'll do this with one-hot encoding.

In [12]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [13]:
toronto_onehot.shape

(1525, 218)

We now group the venues together by neighbourhood, whilst taking the mean for each category.

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.010989,0.021978,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.043956,0.000000,0.000000,0.000,0.010989,0.000000,0.032967,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021978,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.043956,0.000000,0.000000,0.021978,0.000000,0.000000,0.010989,0.032967,0.098901,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.000000,0.010989,0.00000,0.000000,0.000,0.010989,0.000000,0.000000,0.000000,0.000000,0.021978,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.00,0.021978,0.010989,0.000000,0.010989,0.000000,0.000000,0.00,0.010989,0.010989,0.010989,0.00,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.010989,0.000000,0.00000,0.00,0.000000,0.00,0.010989,0.010989,0.010989,0.010989,0.000000

In [15]:
toronto_grouped.shape

(39, 218)

We would like to see the most common venues for each neighbourhood, so we define the following function and apply it to a new dataframe.

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Seafood Restaurant,Bakery,Restaurant,Japanese Restaurant,Beer Bar,Cocktail Bar,Deli / Bodega
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Accessories Store,Diner,Supermarket,Chiropractor,Cocktail Bar
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Restaurant,Italian Restaurant,Yoga Studio,Martial Arts School,Salad Place,Breakfast Spot,Sushi Restaurant,Bank,Bookstore
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Bar,Italian Restaurant,Café,Gym / Fitness Center,Speakeasy,Park,French Restaurant,Grocery Store,Restaurant
4,Central Bay Street,Coffee Shop,Café,Bubble Tea Shop,Pizza Place,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Department Store,Sushi Restaurant,Burger Joint


We now build a K means clustering model with 5 clusters and apply it to our data.

In [18]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[:10]

array([4, 4, 4, 4, 4, 0, 4, 4, 4, 4], dtype=int32)

We now include the cluster labels for each neighbourhood with the most common venues.

In [19]:
neighbourhoods_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

toronto_merged = toronto_data

toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.6784,-79.2941,4,Pub,Gastropub,Neighborhood,Health Food Store,Trail,Bakery,Music Venue,Music Store,Museum,Accessories Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538,4,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Coffee Shop,Dessert Shop,Spa,Cocktail Bar,Cosmetics Shop,Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.6693,-79.3155,4,Fast Food Restaurant,Park,Liquor Store,Coffee Shop,Fish & Chips Shop,Board Shop,Brewery,Movie Theater,Restaurant,Gym
3,M4M,East Toronto,Studio District,43.6561,-79.3406,4,Gym,Diner,Department Store,Garden Center,Coworking Space,Coffee Shop,Baseball Field,Park,Neighborhood,Nightclub
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935,3,Photography Studio,Park,Accessories Store,Nightclub,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum


Finally, let's now rebuild our map, with the colouring in accordance with the cluster label for each neighbourhood.

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster labels']):
    label = folium.Popup(f"{str(poi)} Cluster {str(cluster)}", parse_html=True)
    folium.CircleMarker( 
        [lat, lon], 
        radius=5, 
        popup=label, 
        color=rainbow[cluster - 1],
        fill=True, 
        fill_color=rainbow[cluster - 1], 
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

If we now collect the neighbourhoods within each cluster we can infer the type neighbourhoods it contains. Cluster 0 contains a large number of playgrounds, parks and amenities suggest that these are (higher cost) residential neighbourhoods.

In [21]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"North Toronto West, Lawrence Park",0,Playground,Garden,Park,Gym Pool,Nightclub,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target
8,"Moore Park, Summerhill East",0,Playground,Park,Grocery Store,Gym,Tennis Court,Thai Restaurant,Moving Target,Movie Theater,Moroccan Restaurant,Music Store
10,Rosedale,0,Playground,Candy Store,Park,Grocery Store,Noodle House,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum
19,"Harbourfront East, Union Station, Toronto Islands",0,Café,Harbor / Marina,Park,Music Venue,Accessories Store,Noodle House,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target
30,Christie,0,Café,Grocery Store,Baby Store,Candy Store,Coffee Shop,Park,Playground,Performing Arts Venue,Pet Store,Pizza Place


As the name suggests, this cluster is predominantly a park neighbourhood.

In [22]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,"High Park, The Junction South",1,Park,Accessories Store,Noodle House,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store


The abundance of beauty services, parks, monuments and museums suggest that this is a highly affluent cluster.

In [23]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,2,Health & Beauty Service,Fast Food Restaurant,Accessories Store,Noodle House,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store
23,"Forest Hill North & West, Forest Hill Road Park",2,Health & Beauty Service,Park,Trail,Business Service,Accessories Store,Nightclub,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater


This cluster contains a number of expensive shops and restaurants, but few shops, so could be an affluent suburb.

In [24]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,3,Photography Studio,Park,Accessories Store,Nightclub,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum


This cluster is by far the most populated by less affluent neighbourhoods with more stores and local eating locations.

In [25]:
toronto_merged.loc[toronto_merged['Cluster labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,4,Pub,Gastropub,Neighborhood,Health Food Store,Trail,Bakery,Music Venue,Music Store,Museum,Accessories Store
1,"The Danforth West, Riverdale",4,Greek Restaurant,Italian Restaurant,Café,Ice Cream Shop,Coffee Shop,Dessert Shop,Spa,Cocktail Bar,Cosmetics Shop,Restaurant
2,"India Bazaar, The Beaches West",4,Fast Food Restaurant,Park,Liquor Store,Coffee Shop,Fish & Chips Shop,Board Shop,Brewery,Movie Theater,Restaurant,Gym
3,Studio District,4,Gym,Diner,Department Store,Garden Center,Coworking Space,Coffee Shop,Baseball Field,Park,Neighborhood,Nightclub
5,Davisville North,4,Food & Drink Shop,Department Store,Gym,Breakfast Spot,Dog Run,Gym / Fitness Center,Park,Music Venue,New American Restaurant,Neighborhood
7,Davisville,4,Dessert Shop,Café,Italian Restaurant,Coffee Shop,Park,Sandwich Place,Diner,Gym,Gas Station,Seafood Restaurant
9,"Summerhill West, Rathnelly, South Hill, Forest...",4,Light Rail Station,Coffee Shop,Supermarket,Liquor Store,Accessories Store,New American Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target
11,"St. James Town, Cabbagetown",4,Café,Coffee Shop,Bakery,Chinese Restaurant,Restaurant,Pizza Place,Italian Restaurant,Park,Gourmet Shop,Caribbean Restaurant
12,Church and Wellesley,4,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gay Bar,Yoga Studio,Mediterranean Restaurant,Hotel,Men's Store
13,"Regent Park, Harbourfront",4,Coffee Shop,Breakfast Spot,Yoga Studio,Event Space,Distribution Center,Food Truck,Gym / Fitness Center,Thai Restaurant,Theater,Restaurant
